In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import gc


In [3]:
data = pd.read_csv('included_cohort.csv.gz')

/tmp/ipykernel_1926485/3124945502.py:1: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,19,20,21,22,23,24,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('included_cohort.csv.gz')


In [4]:
len(data)

7499081

In [5]:
data.shape

(7499081, 84)

In [6]:
data.columns

Index(['NIA', 'sexe', 'data_naixement', 'abs_c', 'abs', 'pais_c',
       'N_vaccine_total', 'VACUNA_1_DATA', 'VACUNA_1_MOTIU', 'VACUNA_2_DATA',
       'VACUNA_2_MOTIU', 'VACUNA_3_DATA', 'VACUNA_3_MOTIU', 'VACUNA_1_DATA_pp',
       'VACUNA_2_DATA_pp', 'VACUNA_3_DATA_pp', 'DATA_DM_min', 'DM',
       'covid_bef_vax', 'test_date', 'test_res', 'test_date_covid_2',
       'test_res_covid_2', 'test_date_covid_3', 'test_res_covid_3',
       'test_date_imc_1', 'test_res_imc_1', 'test_date_imc_2',
       'test_res_imc_2', 'test_date_imc_3', 'test_res_imc_3', 'test_date_sp_1',
       'test_res_sp_1', 'test_date_sp_2', 'test_res_sp_2', 'test_date_sp_3',
       'test_res_sp_3', 'test_date_dp_1', 'test_res_dp_1', 'test_date_dp_2',
       'test_res_dp_2', 'test_date_dp_3', 'test_res_dp_3', 'test_date_abdo_1',
       'test_res_abdo_1', 'test_date_abdo_2', 'test_res_abdo_2',
       'test_date_abdo_3', 'test_res_abdo_3', 'test_date_bg_1',
       'test_res_bg_1', 'test_date_bg_2', 'test_res_bg_2', 'test_

In [7]:
data.columns = ['NIA', 'sexe', 'data_naixement', 'abs_c', 'abs', 'pais_c',
       'N_vaccine_total', 'VACUNA_1_DATA', 'VACUNA_1_MOTIU', 'VACUNA_2_DATA',
       'VACUNA_2_MOTIU', 'VACUNA_3_DATA', 'VACUNA_3_MOTIU', 'VACUNA_1_DATA_pp',
       'VACUNA_2_DATA_pp', 'VACUNA_3_DATA_pp', 'DATA_DM_min', 'DM',
       'covid_bef_vax', 'test_date_covid_1', 'test_res_covid_1', 'test_date_covid_2',
       'test_res_covid_2', 'test_date_covid_3', 'test_res_covid_3',
       'test_date_imc_1', 'test_res_imc_1', 'test_date_imc_2',
       'test_res_imc_2', 'test_date_imc_3', 'test_res_imc_3', 'test_date_sp_1',
       'test_res_sp_1', 'test_date_sp_2', 'test_res_sp_2', 'test_date_sp_3',
       'test_res_sp_3', 'test_date_dp_1', 'test_res_dp_1', 'test_date_dp_2',
       'test_res_dp_2', 'test_date_dp_3', 'test_res_dp_3', 'test_date_abdo_1',
       'test_res_abdo_1', 'test_date_abdo_2', 'test_res_abdo_2',
       'test_date_abdo_3', 'test_res_abdo_3', 'test_date_bg_1',
       'test_res_bg_1', 'test_date_bg_2', 'test_res_bg_2', 'test_date_bg_3',
       'test_res_bg_3', 'test_date_chol_1', 'test_res_chol_1',
       'test_date_chol_2', 'test_res_chol_2', 'test_date_chol_3',
       'test_res_chol_3', 'test_date_smoking_1', 'test_res_smoking_1',
       'test_date_smoking_2', 'test_res_smoking_2', 'test_date_smoking_3',
       'test_res_smoking_3', 'test_date_gma_1', 'test_res_gma_1',
       'test_date_gma_2', 'test_res_gma_2', 'test_date_gma_3',
       'test_res_gma_3', 'test_date_sociostat_1', 'test_res_sociostat_1',
       'test_date_sociostat_2', 'test_res_sociostat_2',
       'test_date_sociostat_3', 'test_res_sociostat_3', 'age_1', 'age_2',
       'age_3', 'idabs', 'ISC reescalat']

In [8]:
# Create useful lists

In [9]:
time_covars = ['test_date_covid_1', 'test_res_covid_1', 'test_date_covid_2',
'test_res_covid_2', 'test_date_covid_3', 'test_res_covid_3',
'test_date_imc_1', 'test_res_imc_1', 'test_date_imc_2',
'test_res_imc_2', 'test_date_imc_3', 'test_res_imc_3', 'test_date_sp_1',
'test_res_sp_1', 'test_date_sp_2', 'test_res_sp_2', 'test_date_sp_3',
'test_res_sp_3', 'test_date_dp_1', 'test_res_dp_1', 'test_date_dp_2',
'test_res_dp_2', 'test_date_dp_3', 'test_res_dp_3', 'test_date_abdo_1',
'test_res_abdo_1', 'test_date_abdo_2', 'test_res_abdo_2',
'test_date_abdo_3', 'test_res_abdo_3', 'test_date_bg_1',
'test_res_bg_1', 'test_date_bg_2', 'test_res_bg_2', 'test_date_bg_3',
'test_res_bg_3', 'test_date_chol_1', 'test_res_chol_1',
'test_date_chol_2', 'test_res_chol_2', 'test_date_chol_3',
'test_res_chol_3', 'test_date_smoking_1', 'test_res_smoking_1',
'test_date_smoking_2', 'test_res_smoking_2', 'test_date_smoking_3',
'test_res_smoking_3', 'test_date_gma_1', 'test_res_gma_1',
'test_date_gma_2', 'test_res_gma_2', 'test_date_gma_3',
'test_res_gma_3', 'test_date_sociostat_1', 'test_res_sociostat_1',
'test_date_sociostat_2', 'test_res_sociostat_2',
'test_date_sociostat_3', 'test_res_sociostat_3']

In [10]:
tests = list(set([x.split('_')[2] for x in time_covars]))

In [11]:
# Preprocess

In [12]:
data.test_res_covid_1 = data.test_res_covid_1.map({'Positiu':'Positiu', 'Negatiu': 'Negatiu', 'Probable': 'Positiu', 'No valorat': 'Negatiu'})
data.test_res_covid_1 = data.test_res_covid_1.map({'Positiu': int(1), 'Negatiu': int(0)})
#data.test_res_covid_1 = data.test_res_covid_1.astype(int)

data.test_res_covid_2 = data.test_res_covid_2.map({'Positiu':'Positiu', 'Negatiu': 'Negatiu', 'Probable': 'Positiu', 'No valorat': 'Negatiu'})
data.test_res_covid_2 = data.test_res_covid_2.map({'Positiu': int(1), 'Negatiu': int(0)})

data.test_res_covid_3 = data.test_res_covid_3.map({'Positiu':'Positiu', 'Negatiu': 'Negatiu', 'Probable': 'Positiu', 'No valorat': 'Negatiu'})
data.test_res_covid_3 = data.test_res_covid_3.map({'Positiu': int(1), 'Negatiu': int(0)})

In [13]:
data['Vacuna_1'] = np.zeros(len(data))
data['Vacuna_2'] = np.zeros(len(data))
data['Vacuna_3'] = np.zeros(len(data))

data.loc[~data.VACUNA_1_DATA.isna(), 'Vacuna_1'] = int(1)
data.loc[~data.VACUNA_2_DATA.isna(), 'Vacuna_2'] = int(1)
data.loc[~data.VACUNA_3_DATA.isna(), 'Vacuna_3'] = int(1)

In [14]:
data.head()

,NIA,sexe,data_naixement,abs_c,abs,pais_c,N_vaccine_total,VACUNA_1_DATA,VACUNA_1_MOTIU,VACUNA_2_DATA,...,test_date_sociostat_3,test_res_sociostat_3,age_1,age_2,age_3,idabs,ISC reescalat,Vacuna_1,Vacuna_2,Vacuna_3
0,5694814,H,1992-07-18T00:00:00.000Z,286.0,Granollers 3 centre est,NaN,0,NaN,NaN,NaN,...,2021-01-01T00:00:00.000Z,2.0,28.890411,28.972603,29.057534,286.0,29.305315,0.0,0.0,0.0
1,43585684,H,1987-08-31T00:00:00.000Z,259.0,Tremp,724.0,0,NaN,NaN,NaN,...,2021-01-01T00:00:00.000Z,2.0,33.775342,33.857534,33.942466,259.0,46.763759,0.0,0.0,0.0
2,24884898,H,2000-04-08T00:00:00.000Z,22.0,Barcelona 2-B,12.0,0,NaN,NaN,NaN,...,2021-01-01T00:00:00.000Z,3.0,21.161644,21.243836,21.328767,22.0,19.174582,0.0,0.0,0.0
3,10107675,H,1988-11-03T00:00:00.000Z,257.0,Torredembarra,NaN,0,NaN,NaN,NaN,...,2021-01-01T00:00:00.000Z,3.0,32.597260,32.679452,32.764384,257.0,41.018410,0.0,0.0,0.0
4,22995342,D,1990-12-29T00:00:00.000Z,22.0,Barcelona 2-B,620.0,0,NaN,NaN,NaN,...,2021-01-01T00:00:00.000Z,2.0,30.443836,30.526027,30.610959,22.0,19.174582,0.0,0.0,0.0


In [ ]:
### Check NaNs

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'column_name': data.columns,'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)


In [15]:
missing_value_df.head(42)

,column_name,percent_missing
NIA,NIA,0.000000
Vacuna_1,Vacuna_1,0.000000
Vacuna_2,Vacuna_2,0.000000
covid_bef_vax,covid_bef_vax,0.000000
DM,DM,0.000000
N_vaccine_total,N_vaccine_total,0.000000
Vacuna_3,Vacuna_3,0.000000
data_naixement,data_naixement,0.000000
sexe,sexe,0.000000
abs_c,abs_c,0.000013


In [16]:
missing_value_df.tail(42)

,column_name,percent_missing
test_date_imc_3,test_date_imc_3,31.717353
test_res_imc_3,test_res_imc_3,31.785335
test_date_imc_2,test_date_imc_2,32.183704
test_res_imc_2,test_res_imc_2,32.251979
test_date_imc_1,test_date_imc_1,32.429987
test_res_imc_1,test_res_imc_1,32.498102
test_date_smoking_3,test_date_smoking_3,40.677438
test_res_smoking_3,test_res_smoking_3,40.677438
VACUNA_3_DATA_pp,VACUNA_3_DATA_pp,40.857660
test_res_smoking_2,test_res_smoking_2,41.140534


In [12]:
### Check covariate variety and nans

In [16]:
### Variety

In [17]:
for test_i in tests:
  cols = ['test_date_{}_1'.format(test_i), 'test_date_{}_2'.format(test_i), 'test_date_{}_3'.format(test_i)]
  data[test_i] = data[cols].apply(lambda x: sum( [x[cols[0]]==x[cols[1]], x[cols[0]]==x[cols[2]], x[cols[1]]==x[cols[2]]] ), axis=1)

In [18]:
for test in tests:
  print(data[test].value_counts())

sp
3    4534088
0    2189567
1     775426
Name: count, dtype: int64
sociostat
3    4573326
1    2132596
0     793159
Name: count, dtype: int64
abdo
0    7055621
3     430187
1      13273
Name: count, dtype: int64
chol
0    4013406
3    3485596
1         79
Name: count, dtype: int64
smoking
3    4357858
0    3085519
1      55704
Name: count, dtype: int64
gma
3    4573326
1    2132596
0     793159
Name: count, dtype: int64
imc
3    4541895
0    2444032
1     513154
Name: count, dtype: int64
covid
0    3954341
3    2337201
1    1207539
Name: count, dtype: int64
bg
0    3943544
3    3552977
1       2560
Name: count, dtype: int64
dp
3    4532960
0    2190918
1     775203
Name: count, dtype: int64


In [19]:
### Nans

In [20]:
for test_i in tests:
  cols = ['test_date_{}_1'.format(test_i), 'test_date_{}_2'.format(test_i), 'test_date_{}_3'.format(test_i)]
  data['{}_notcarried'.format(test_i)] = data[cols].apply(lambda x: 1 if pd.isna(x[cols[0]])
                                                                    else ( 2 if (pd.isna(x[cols[1]]))
                                                                    else ( 3 if (pd.isna(x[cols[1]]))
                                                                    else 4)), axis=1)

In [21]:
for test in tests:
  print(data['{}_notcarried'.format(test_i)].value_counts())

dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64
dp_notcarried
4    5364278
1    2134803
Name: count, dtype: int64


In [12]:
### Outlier removal

In [15]:
# IMC above 70 becomes NaN
data.loc[data.test_res_imc_1>70, 'test_res_imc_1'] = np.NaN
data.loc[data.test_res_imc_2>70, 'test_res_imc_2'] = np.NaN
data.loc[data.test_res_imc_2>70, 'test_res_imc_2'] = np.NaN

In [16]:
# People of more than 110 years must be dead
data.data_naixement = data.data_naixement.str[0:4].astype(int)
data = data[data.data_naixement>1914]

In [17]:
### Baseline cov NaN imputation (based on other covariates)

In [18]:
# Time-invariant pre-baseline covars (with nans): country of origin, abs, isc. Sample from prob. distrib, with frequencies equal to those in the present data. 
# In this way, we end up with the value that is "most likely", and ¿errors will cancel each other out?

In [19]:
pais_c_sampled = np.random.choice(data.pais_c.value_counts().index.values, size = len(data.pais_c.isna()), p = data.pais_c.value_counts().values/sum(data.pais_c.value_counts().values))

In [20]:
data.loc[data.pais_c.isna(), 'pais_c'] = pd.Series(pais_c_sampled)

In [21]:
abs_sampled = np.random.choice(data.abs_c.value_counts().index.values, size = len(data.abs_c.isna()), p = data.abs_c.value_counts().values/sum(data.abs_c.value_counts().values))

In [22]:
data.loc[data.abs_c.isna(), 'abs_c'] = pd.Series(abs_sampled)

In [23]:
# Time-varying baseline covars 

In [24]:
dag_list = ['BMI -> Blood_pressure_dp',
'BMI -> Blood_pressure_sp',
'BMI -> Charlson_i',
'BMI -> Covid_infection',
'BMI -> Pre_diabetes',
'BMI -> Waist_size',
'Birth_year -> BMI',
'Birth_year -> Blood_pressure_dp',
'Birth_year -> Blood_pressure_sp',
'Birth_year -> Charlson_i',
'Birth_year -> Cholesterol',
'Birth_year -> Pre_diabetes',
'Birth_year -> Smoking',
'Birth_year -> Socio_status',
'Birth_year -> Waist_size',
'Blood_pressure_dp -> Charlson_i',
'Blood_pressure_sp -> Charlson_i',
'Charlson_i -> Covid_infection',
'Cholesterol -> Charlson_i',
'Country_origin -> BMI',
'Country_origin -> Blood_pressure_dp',
'Country_origin -> Blood_pressure_sp',
'Country_origin -> Charlson_i',
'Country_origin -> Cholesterol',
'Country_origin -> Pre_diabetes',
'Country_origin -> Smoking',
'Country_origin -> Socio_status',
'Country_origin -> Waist_size',
'Pre_diabetes -> Charlson_i',
'Pre_diabetes -> Covid_infection',
'Smoking -> BMI',
'Smoking -> Blood_pressure_dp',
'Smoking -> Blood_pressure_sp',
'Smoking -> Charlson_i',
'Smoking -> Covid_infection',
'Smoking -> Waist_size',
'Socio_status -> BMI',
'Socio_status -> Blood_pressure_dp',
'Socio_status -> Blood_pressure_sp',
'Socio_status -> Cholesterol',
'Socio_status -> Covid_infection',
'Socio_status -> Pre_diabetes',
'Socio_status -> Smoking']

In [25]:
pair_list = []
for pair in dag_list:
  pair_list.append(pair.split(' -> '))

In [26]:
pair_list = pd.DataFrame(pair_list).replace({'BMI': 'imc',
 'Birth_year': 'data_naixement',
 'Waist_size':  'abdo',
 'Cholesterol':  'chol',
 'Socio_status':  'sociostat',
 'Charlson_i':  'gma',
 'Blood_pressure_dp':  'dp',
 'Blood_pressure_sp':  'sp',
 'Covid_infection': 'covid',
 'Pre_diabetes': 'bg',
 'Country_origin': 'pais_c',
 'Smoking': 'smoking'}).values.tolist()

In [27]:
### Imputting with random forest strategy

In [28]:
# Test result

In [29]:
# 'Flatten' list 
flat_list = [x for inner_list in pair_list for x in inner_list]

for elem in set(flat_list)-set(['pais_c', 'data_naixement']):
  print(elem)
  affecting_vars = []
  for pair in pair_list:
    if pair[1]==elem:
      #print(pair)
      affecting_vars.append(pair[0])
  print(affecting_vars)
    
  affecting_vars_fullname = []
  for var in affecting_vars:
    #print(var)
    if ((var!='pais_c') and (var!='data_naixement')):
      affecting_vars_fullname.append('test_res_{}_1'.format(var))
    else:
      affecting_vars_fullname.append(var)


smoking
['data_naixement', 'pais_c', 'sociostat']
sp
['imc', 'data_naixement', 'pais_c', 'smoking', 'sociostat']
chol
['data_naixement', 'pais_c', 'sociostat']
abdo
['imc', 'data_naixement', 'pais_c', 'smoking']
bg
['imc', 'data_naixement', 'pais_c', 'sociostat']
covid
['imc', 'gma', 'bg', 'smoking', 'sociostat']
dp
['imc', 'data_naixement', 'pais_c', 'smoking', 'sociostat']
sociostat
['data_naixement', 'pais_c']
imc
['data_naixement', 'pais_c', 'smoking', 'sociostat']
gma
['imc', 'data_naixement', 'dp', 'sp', 'chol', 'pais_c', 'bg', 'smoking']


In [30]:
# 'Flatten' list 
flat_list = [x for inner_list in pair_list for x in inner_list]

for elem in set(flat_list)-set(['pais_c', 'data_naixement']):
  print(elem)
  affecting_vars = []
  for pair in pair_list:
    if pair[1]==elem:
      #print(pair)
      affecting_vars.append(pair[0])
  #print(affecting_vars)
    
  affecting_vars_fullname = []
  for var in affecting_vars:
    #print(var)
    if ((var!='pais_c') and (var!='data_naixement')):
      affecting_vars_fullname.append('test_res_{}_1'.format(var))
    else:
      affecting_vars_fullname.append(var)
  
  if (elem=='covid')|(elem=='sociostat'):
    regr = RandomForestClassifier(n_jobs=36)
    regr.fit(data.loc[~data['test_res_{}_1'.format(elem)].isna(), affecting_vars_fullname], data.loc[~data['test_res_{}_1'.format(elem)].isna(), 'test_res_{}_1'.format(elem)])
  else:
    if elem=='gma':
      njobs=18
    else:
      njobs=36
    regr = RandomForestRegressor(n_jobs=njobs)
    regr.fit(data.loc[~data['test_res_{}_1'.format(elem)].isna(), affecting_vars_fullname], data.loc[~data['test_res_{}_1'.format(elem)].isna(), 'test_res_{}_1'.format(elem)])
    gc.collect()
    
  data.loc[data['test_res_{}_1'.format(elem)].isna(), 'test_res_{}_1'.format(elem)] = regr.predict(data.loc[data['test_res_{}_1'.format(elem)].isna(), affecting_vars_fullname])

gc.collect()

smoking
sp
chol
abdo
bg
covid
classif
dp
sociostat
classif
imc
gma


In [ ]:
# Problem: GMA blocks the whole memory and gives memory errors

In [31]:
# Test date. Input all at 2021-01-01. Is this the best approach?

In [32]:
for test in tests:
  data['test_date_{}_1'.format(test)] = data['test_date_{}_1'.format(test)].fillna(value='2021-01-01T00:00:00.000Z')

In [33]:
### Carry-forward

In [34]:
for test in tests:
  for i in range(2,4):
    data.loc[data['test_res_{}_{}'.format(test,i)].isna(), 'test_res_{}_{}'.format(test,i)] = data.loc[data['test_res_{}_{}'.format(test,i)].isna(), 'test_res_{}_{}'.format(test,i-1)]
    data.loc[data['test_date_{}_{}'.format(test,i)].isna(), 'test_date_{}_{}'.format(test,i)] = data.loc[data['test_date_{}_{}'.format(test,i)].isna(), 'test_date_{}_{}'.format(test,i-1)]

In [51]:
data.isna().any().head(50)

NIA                  False
sexe                 False
data_naixement       False
abs_c                False
abs                   True
pais_c               False
N_vaccine_total      False
VACUNA_1_DATA         True
VACUNA_1_MOTIU        True
VACUNA_2_DATA         True
VACUNA_2_MOTIU        True
VACUNA_3_DATA         True
VACUNA_3_MOTIU        True
VACUNA_1_DATA_pp      True
VACUNA_2_DATA_pp      True
VACUNA_3_DATA_pp      True
DATA_DM_min           True
DM                   False
covid_bef_vax        False
test_date_covid_1    False
test_res_covid_1     False
test_date_covid_2    False
test_res_covid_2     False
test_date_covid_3    False
test_res_covid_3     False
test_date_imc_1      False
test_res_imc_1       False
test_date_imc_2      False
test_res_imc_2       False
test_date_imc_3      False
test_res_imc_3       False
test_date_sp_1       False
test_res_sp_1        False
test_date_sp_2       False
test_res_sp_2        False
test_date_sp_3       False
test_res_sp_3        False
t

In [52]:
data.isna().any().tail(50)

test_date_dp_1           False
test_res_dp_1            False
test_date_dp_2           False
test_res_dp_2            False
test_date_dp_3           False
test_res_dp_3            False
test_date_abdo_1         False
test_res_abdo_1          False
test_date_abdo_2         False
test_res_abdo_2          False
test_date_abdo_3         False
test_res_abdo_3          False
test_date_bg_1           False
test_res_bg_1            False
test_date_bg_2           False
test_res_bg_2            False
test_date_bg_3           False
test_res_bg_3            False
test_date_chol_1         False
test_res_chol_1          False
test_date_chol_2         False
test_res_chol_2          False
test_date_chol_3         False
test_res_chol_3          False
test_date_smoking_1      False
test_res_smoking_1       False
test_date_smoking_2      False
test_res_smoking_2       False
test_date_smoking_3      False
test_res_smoking_3       False
test_date_gma_1          False
test_res_gma_1           False
test_dat

In [37]:
# Some final checks

In [38]:
### Save

In [39]:
data.to_csv('included_cohort_prep.csv')

In [40]:
### Save sample

In [41]:
data_sample = data.sample(frac=0.05, random_state=1)

In [42]:
data_sample.to_csv('included_cohort_sample_prep.csv')

In [43]:
# Some final checks

### Trash

In [120]:
# 'Flatten' list 
flat_list = [x for inner_list in pair_list for x in inner_list]

for elem in set(flat_list):
  #print(elem)
  affecting_vars = []
  for pair in pair_list:
    if pair[1]==elem:
      #print(pair)
      affecting_vars.append(pair[0])
      
  test_res_list = ['data_naixement', 'pais_c']
  for var in list(set(affecting_vars)-set(['data_naixement', 'pais_c'])):
    for i in range(1,4):
      test_res_list.append('test_res_{}_{}'.format(var, i))
  
  data_affecting_vars = data[test_res_list]
  
  # Sample from normal distribution with mean of the missing var stratified by affecting vars
  for i in range(1,4):
    for index, row in data[data['test_res_{}_{}'.format(elem,i)].isna()].iterrows():
      data.loc[index, 'test_res_{}_{}'.format(elem,i)] = data[data_affecting_vars.row==row, 'test_res_{}_{}'.format(elem,i)].mean(skipna=True)


AttributeError: 'DataFrame' object has no attribute 'row'

In [ ]:
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)
print(regr.predict([[0, 0, 0, 0]]))

In [96]:
data[test_res_list].isna().any()

data_naixement          False
pais_c                  False
test_res_sociostat_1     True
test_res_sociostat_2     True
test_res_sociostat_3     True
test_res_smoking_1       True
test_res_smoking_2       True
test_res_smoking_3       True
test_res_imc_1           True
test_res_imc_2           True
test_res_imc_3           True
dtype: bool

In [80]:
test_res_list = []
for var in tests:
  for i in range(1,4):
    test_res_list.append('test_res_{}_{}'.format(var, i))

In [97]:
test_res_list

['data_naixement',
 'pais_c',
 'test_res_sociostat_1',
 'test_res_sociostat_2',
 'test_res_sociostat_3',
 'test_res_smoking_1',
 'test_res_smoking_2',
 'test_res_smoking_3',
 'test_res_imc_1',
 'test_res_imc_2',
 'test_res_imc_3']

In [66]:
import numpy as np

from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=10, random_state=0)

data_tofit = [[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]]

imp.fit(data_tofit)
IterativeImputer(random_state=0)

X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]

# the model learns that the second feature is double the first

print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


In [68]:
X_test

[[nan, 2], [6, nan], [nan, 6]]